In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
URL = "https://www.voetbalstats.nl/listjaarere.php?seizoenid=02"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
td_elements = soup.find_all("td", class_="linkere")

In [7]:
wedstrijden = []
for td in td_elements:
    a_tag = td.find("a", href=True)
    if a_tag and a_tag["href"].startswith("opstelere"):
        wedstrijden.append(a_tag["href"])

wedstrijden = list(set(wedstrijden))
print(len(wedstrijden))

ids = [int(x.split("=", 1)[1]) for x in wedstrijden]
print(wedstrijden)

306
['opstelere.php?wid=95', 'opstelere.php?wid=283', 'opstelere.php?wid=194', 'opstelere.php?wid=157', 'opstelere.php?wid=201', 'opstelere.php?wid=252', 'opstelere.php?wid=243', 'opstelere.php?wid=92', 'opstelere.php?wid=79', 'opstelere.php?wid=56', 'opstelere.php?wid=262', 'opstelere.php?wid=7', 'opstelere.php?wid=127', 'opstelere.php?wid=276', 'opstelere.php?wid=132', 'opstelere.php?wid=220', 'opstelere.php?wid=102', 'opstelere.php?wid=228', 'opstelere.php?wid=77', 'opstelere.php?wid=242', 'opstelere.php?wid=286', 'opstelere.php?wid=203', 'opstelere.php?wid=253', 'opstelere.php?wid=2', 'opstelere.php?wid=209', 'opstelere.php?wid=88', 'opstelere.php?wid=146', 'opstelere.php?wid=140', 'opstelere.php?wid=42', 'opstelere.php?wid=245', 'opstelere.php?wid=80', 'opstelere.php?wid=211', 'opstelere.php?wid=126', 'opstelere.php?wid=87', 'opstelere.php?wid=135', 'opstelere.php?wid=199', 'opstelere.php?wid=15', 'opstelere.php?wid=59', 'opstelere.php?wid=299', 'opstelere.php?wid=160', 'opstelere

In [5]:
maanden = {"januari" : 1, "februari": 2, "maart": 3, "april": 4, "mei": 5, "juni": 6, "juli": 7, "augustus": 8, "september": 9, "oktober": 10, "november": 11, "december": 12}
weekdagen = {"maandag": 1, "dinsdag": 2, "woensdag": 3, "donderdag": 4, "vrijdag": 5, "zaterdag": 6, "zondag": 7}

def get_datum(datum):
    weekdag  = int(weekdagen[datum.split()[0]])
    dag = int(datum.split()[1])
    maand = int(maanden[datum.split()[2]])
    jaar = int(datum.split()[3])

    return date(jaar, maand, dag)

In [6]:
def get_minuut(doelpunt):
    minuut = doelpunt.strip().split(". ")[0]
    try:
        return int(minuut)
    except:
        return minuut


In [50]:
rows = []
wedstrijd_soups = []

for i in range(len(wedstrijden)):
    wedstrijd_URL = "https://www.voetbalstats.nl/" + wedstrijden[i]
    id = ids[i]

    page = requests.get(wedstrijd_URL)
    soup = BeautifulSoup(page.content, "html.parser")

    wedstrijd_soups.append(soup)

    page_title = soup.title.string if soup.title else "No title found"
    thuis = page_title.split("-")[0].strip()
    uit = page_title.split("-")[1].split("(")[0].strip()
    print(i, len(wedstrijden), ids[0], page_title)

    td_elements2 = soup.find_all("td", class_="linkere")
    td_elements3 = soup.find_all('td', {'align': 'center', 'width': '10%', 'class': 'nl3-wizw'})

    scores = []
    for td in td_elements3:
        score = td.text.strip().replace(' ', '')  # Removes spaces and trims leading/trailing whitespace
        scores.append(score)


    scores_list = []

    for i in range(1, len(scores)):
        if scores[i].split("\r")[0][0] > scores[i-1].split("\r")[0][0]:
            team = thuis
        else:
            team = uit
        scores_list.append([scores[i].split("\r")[0], team])

    datum = ""
    for td in td_elements2:
        a_tag = td.find("a", href=True)
        if a_tag and a_tag["href"].startswith("listjaarere"):
            datum = get_datum(a_tag.contents[0])


    a_tags = soup.find_all("a")

    spelers = []

    for a_tag in a_tags:
        bal_tag = a_tag.find('img', src='./afbeeldingen/bal.jpg')
        balpen_tag = a_tag.find('img', src='./afbeeldingen/balpen.jpg')
        baled_tag = a_tag.find('img', src='./afbeeldingen/baled.jpg')
        if bal_tag:
            spelers.append(["doelpunt", a_tag.contents[1]])
        if balpen_tag:
            spelers.append(["penalty",a_tag.contents[2]])
        if baled_tag:
            spelers.append(["eigen_doelpunt",a_tag.contents[2]])



    for i in range(len(spelers)):
        speler = spelers[i]
        soort = speler[0]
        minuut = get_minuut(speler[1].strip().split(". ")[0])
        naam = speler[1].strip().split(". ")[1]
        score = scores_list[i]

        rows.append([id, datum, thuis, uit, minuut, soort, naam, score[0], score[1]])


0 306 103 Willem II - NAC (07-04-1957 - Voetbalstats.nl)
1 306 103 Fortuna '54 - Willem II (24-02-1957 - Voetbalstats.nl)
2 306 103 Rapid JC - NAC (11-11-1956 - Voetbalstats.nl)
3 306 103 BVC Amsterdam - VVV (09-12-1956 - Voetbalstats.nl)
4 306 103 Feijenoord - Willem II (20-01-1957 - Voetbalstats.nl)
5 306 103 Sparta - VVV (13-01-1957 - Voetbalstats.nl)
6 306 103 SC Enschede - BVV (07-10-1956 - Voetbalstats.nl)
7 306 103 BVC Amsterdam - SC Enschede (21-10-1956 - Voetbalstats.nl)
8 306 103 Elinkwijk - Feijenoord (19-05-1957 - Voetbalstats.nl)
9 306 103 BVC Amsterdam - EVV Eindhoven (24-03-1957 - Voetbalstats.nl)
10 306 103 BVV - Sparta (20-04-1957 - Voetbalstats.nl)
11 306 103 BVV - MVV (06-01-1957 - Voetbalstats.nl)
12 306 103 Sparta - DOS (20-01-1957 - Voetbalstats.nl)
13 306 103 Fortuna '54 - Elinkwijk (10-02-1957 - Voetbalstats.nl)
14 306 103 DOS - NAC (02-06-1957 - Voetbalstats.nl)
15 306 103 Sparta - BVC Amsterdam (07-10-1956 - Voetbalstats.nl)
16 306 103 Fortuna '54 - BVV (28-10

In [52]:
df = pd.DataFrame(rows, columns=["Id", "Datum", "Thuis", "Uit", "Minuut", "Soort", "Speler", "Score", "Team"])
df["Soort"] = df["Soort"].replace("eigen_doelpunt", "eigen doelpunt")
df.to_csv('1956.csv', index=False)  

In [67]:
doelpunten = df[df["Soort"] == "doelpunt"].groupby("Speler").size().reset_index(name="Doelpunt Count").sort_values(by="Doelpunt Count", ascending=False).reset_index(drop=True)
print(doelpunten)

                   Speler  Doelpunt Count
0            Coen  Dillen              41
1        Cor van der Gijp              32
2    Tonny van der Linden              27
3           Dirk  Lammers              20
4           Piet  Fransen              20
..                    ...             ...
168           Jo  Dingena               1
169         John  Geraeds               1
170         Leen  Corbran               1
171   Martinus  Okhuijsen               1
172      Wim van der Waal               1

[173 rows x 2 columns]


In [26]:
rows = []

wedstrijd_URL = "https://www.voetbalstats.nl/" + wedstrijden[0]
id = ids[0]

page = requests.get(wedstrijd_URL)
soup = BeautifulSoup(page.content, "html.parser")

page_title = soup.title.string if soup.title else "No title found"
thuis = page_title.split("-")[0].strip()
uit = page_title.split("-")[1].split("(")[0].strip()
print(ids[0], page_title)

td_elements2 = soup.find_all("td", class_="linkere")
td_elements3 = soup.find_all('td', {'align': 'center', 'width': '10%', 'class': 'nl3-wizw'})

scores = []
for td in td_elements3:
    score = td.text.strip().replace(' ', '')  # Removes spaces and trims leading/trailing whitespace
    scores.append(score)


datum = ""
for td in td_elements2:
    a_tag = td.find("a", href=True)
    if a_tag and a_tag["href"].startswith("listjaarere"):
        datum = get_datum(a_tag.contents[0])


a_tags = soup.find_all("a")

doelpunt_spelers = []
penalty_spelers = []
eigen_doelpunt_spelers = []

for a_tag in a_tags:
    bal_tag = a_tag.find('img', src='./afbeeldingen/bal.jpg')
    balpen_tag = a_tag.find('img', src='./afbeeldingen/balpen.jpg')
    baled_tag = a_tag.find('img', src='./afbeeldingen/baled.jpg')
    if bal_tag:
        doelpunt_spelers.append(a_tag.contents[1])
    if balpen_tag:
        penalty_spelers.append(a_tag.contents[2])
    if baled_tag:
        eigen_doelpunt_spelers.append(a_tag.contents[2])

for speler in doelpunt_spelers:
    minuut = get_minuut(speler.strip().split(". ")[0])
    naam = speler.strip().split(". ")[1]
    rows.append([id, datum, thuis, uit, minuut, "doelpunt", naam])

for speler in eigen_doelpunt_spelers:
    minuut = get_minuut(speler.strip().split(". ")[0])
    naam = speler.strip().split(". ")[1]
    rows.append([id, datum, thuis, uit, minuut, "eigen_doelpunt", naam])

for speler in penalty_spelers:
    minuut = get_minuut(speler.strip().split(". ")[0])
    naam = speler.strip().split(". ")[1]
    rows.append([id, datum, thuis, uit, minuut, "penalty", naam])

71 NAC - Feijenoord (31-03-1957 - Voetbalstats.nl)


In [35]:
scores[1].split("\r")[0]

'0-1'